In [17]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder,StandardScaler, FunctionTransformer, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report, mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, RANSACRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.cross_decomposition import PLSRegression


In [18]:
# Load train and test data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# === Step 1: Outlier removal (ONLY for train data) ===
num_features = train_df.select_dtypes(include='number').drop(columns=['Scoville Heat Units (SHU)']).columns

for col in num_features:
    q_low = train_df[col].quantile(0.001)
    q_high = train_df[col].quantile(0.995)
    train_df = train_df[(train_df[col] >= q_low) & (train_df[col] <= q_high)]

# === Step 2: Drop column from both train and test ===
DROP_COLS = ['Average Temperature During Storage (celcius)']
train_df = train_df.drop(columns=DROP_COLS, errors='ignore')
test_df = test_df.drop(columns=DROP_COLS, errors='ignore')

# === Step 3: Define features ===
num_features = train_df.select_dtypes(include='number').drop(columns=['Scoville Heat Units (SHU)'], errors='ignore').columns
cat_features = ['color', 'Harvest Time']

# === Step 4: Column transformers ===

# Numeric preprocessing: impute + scale
num_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing: impute + encode
cat_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(transformers=[
    ('num', num_preprocessor, num_features),
    ('cat', cat_preprocessor, cat_features)
])

In [19]:
# Splitting data
X = train_df.drop(columns=['Scoville Heat Units (SHU)'])
y = train_df['Scoville Heat Units (SHU)']


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Evaluate function
def evaluate_model(name, model, param_grid):
    print(f"\n Grid Search for {name}")
    
    pipe = Pipeline ([
        ('preprocess', preprocessor),
        ('regressor', model)
    ])
    
    # Adjust param grid to reflect the pipeline's step name
    param_grid = {f'regressor__{key}': val for key, val in param_grid.items()}

    grid = GridSearchCV(pipe, param_grid=param_grid, cv=5,
                        scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train) 

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Best Params: {grid.best_params_}")
    print(f"MSE: {mse:.2f}")
    return best_model

### (A) REGRESSION ANALYSIS

In [21]:
# 1. Linear Regression
lr_model = evaluate_model("Linear Regression", LinearRegression(), {})

# 2. Ridge Regression
ridge_model = evaluate_model("Ridge Regression", Ridge(), {'alpha': [0.01, 0.1, 1, 10]})

# 3. Lasso Regression
lasso_model = evaluate_model("Lasso Regression", Lasso(), {'alpha': [0.01, 0.1, 1, 10]})

# 4. Decision Tree
dt_model = evaluate_model("Decision Tree", DecisionTreeRegressor(random_state=42), {
    'max_depth': [3, 5, 10, None]
})

# 5. Random Forest
rf_model = evaluate_model("Random Forest", RandomForestRegressor(random_state=42), {
    'n_estimators': [50, 100],
    'max_depth': [None, 10]
})

# 6. Gradient Boosting
gb_model = evaluate_model("Gradient Boosting", GradientBoostingRegressor(random_state=42), {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
})

# 7. Support Vector Regressor
svr_model = evaluate_model("Support Vector Regressor", SVR(), {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
})


 Grid Search for Linear Regression
Best Params: {}
MSE: 9319064985.92

 Grid Search for Ridge Regression
Best Params: {'regressor__alpha': 10}
MSE: 9313843111.71

 Grid Search for Lasso Regression


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.762e+11, tolerance: 6.794e+08
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.706e+11, tolerance: 6.790e+08
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Best Params: {'regressor__alpha': 10}
MSE: 9318272681.18

 Grid Search for Decision Tree
Best Params: {'regressor__max_depth': 3}
MSE: 10697709539.18

 Grid Search for Random Forest
Best Params: {'regressor__max_depth': 10, 'regressor__n_estimators': 100}
MSE: 10195450864.96

 Grid Search for Gradient Boosting
Best Params: {'regressor__learning_rate': 0.05, 'regressor__max_depth': 3, 'regressor__n_estimators': 50}
MSE: 9778414323.67

 Grid Search for Support Vector Regressor
Best Params: {'regressor__C': 10, 'regressor__kernel': 'linear'}
MSE: 14909390929.68


In [22]:
poly_model = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('linear', LinearRegression())
])
param_grid = {
    'poly_features__degree': [2, 3],
    'linear__fit_intercept': [True, False]
}
evaluate_model("Polynomial Regression", poly_model, param_grid)


 Grid Search for Polynomial Regression
Best Params: {'regressor__linear__fit_intercept': True, 'regressor__poly_features__degree': 2}
MSE: 11154481992.15


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Length (cm)', 'Width (cm)', 'Weight (g)', 'Pericarp Thickness (mm)',
       'Seed Count', 'Capsaicin Content', 'Vitamin C Content (mg)',
       'Sugar Content', 'Moisture Content', 'Firmness',
       'Average Daily Temperature During Growth (celcius)'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['color', 'Harvest Time'])])),
                ('regressor',
                 Pipeline(steps=[('poly_features', PolynomialFeatures()),
                                 ('linear', LinearRegression())]))])

In [24]:
pcr_model = Pipeline([
    ('pca', PCA()),
    ('linear', LinearRegression())
])
param_grid = {
    'pca__n_components': [0.90, 0.95, 0.99],
    'linear__fit_intercept': [True, False]
}
pcr_model= evaluate_model("PCR", pcr_model, param_grid)



 Grid Search for PCR
Best Params: {'regressor__linear__fit_intercept': True, 'regressor__pca__n_components': 0.9}
MSE: 9314463789.06


In [27]:
pls = PLSRegression()
param_grid = {
    'n_components': [2, 5, 10]
}
pls_model = evaluate_model("PLS Regression", pls, param_grid)



 Grid Search for PLS Regression
Best Params: {'regressor__n_components': 2}
MSE: 9282754172.23


In [29]:
ransac = RANSACRegressor(LinearRegression())
param_grid = {
    'min_samples': [0.5, 0.75],
    'residual_threshold': [5.0, 10.0]
}
evaluate_model("RANSAC Regression", ransac, param_grid)



 Grid Search for RANSAC Regression


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Pyth

Best Params: {'regressor__min_samples': 0.5, 'regressor__residual_threshold': 10.0}
MSE: 10502089697.48


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Library/Frameworks/Pyth

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Length (cm)', 'Width (cm)', 'Weight (g)', 'Pericarp Thickness (mm)',
       'Seed Count', 'Capsaicin Content', 'Vitamin C Content (mg)',
       'Sugar Content', 'Moisture Content', 'Firmness',
       'Average Daily Temperature During Growth (celcius)'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['color', 'Harvest Time'])])),
                ('regressor',
                 RANSACRegressor(estimator=LinearRegression(), min_samples=0.5,
                                 residual_threshold=10.0))])

In [30]:
ridge_params = {'alpha': [0.1, 1.0, 10.0]}
evaluate_model("Ridge Regression", Ridge(), ridge_params)

lasso_params = {'alpha': [0.1, 1.0, 10.0]}
evaluate_model("Lasso Regression", Lasso(), lasso_params)

elastic_params = {'alpha': [0.1, 1.0], 'l1_ratio': [0.2, 0.5, 0.8]}
evaluate_model("ElasticNet", ElasticNet(), elastic_params)


 Grid Search for Ridge Regression
Best Params: {'regressor__alpha': 10.0}
MSE: 9313843111.71

 Grid Search for Lasso Regression


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.762e+11, tolerance: 6.794e+08
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.809e+11, tolerance: 6.900e+08
  model = cd_fast.enet_coordinate_descent(


Best Params: {'regressor__alpha': 10.0}
MSE: 9318272681.18

 Grid Search for ElasticNet
Best Params: {'regressor__alpha': 1.0, 'regressor__l1_ratio': 0.8}
MSE: 9291861587.90


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Length (cm)', 'Width (cm)', 'Weight (g)', 'Pericarp Thickness (mm)',
       'Seed Count', 'Capsaicin Content', 'Vitamin C Content (mg)',
       'Sugar Content', 'Moisture Content', 'Firmness',
       'Average Daily Temperature During Growth (celcius)'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['color', 'Harvest Time'])])),
                ('regressor', ElasticNet(l1_ratio=0.8))])

### (B) Multi-class classification analysis with an ensemble classifier.

In [ ]:
# Define bin count
num_bin = 50
X_binned = X.copy()
y_binned = pd.qcut(y, q=num_bin, labels=False, duplicates='drop')

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X_binned, y_binned, test_size=0.2, random_state=42)

# Build classification pipeline
clf_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Hyperparameter tuning
param_grid_rf = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [5, 10]
}

grid_clf = GridSearchCV(clf_pipeline, param_grid=param_grid_rf, 
                        cv=5, scoring='f1_macro')

grid_clf.fit(X_train, y_train)
print('Best hyperparameters:', grid_clf.best_params_)

# Evaluate
y_pred = grid_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy score:', accuracy)
print(classification_report(y_test, y_pred))

Best hyperparameters: {'classifier__max_depth': 10, 'classifier__n_estimators': 100}
Accuracy score: 0.49732620320855614
              precision    recall  f1-score   support

           0       0.59      1.00      0.74        92
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         4
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         5
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         5
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         4
          13       1.00      0.25      0.40         4
          14       0.00      0.00      0.00         8
          15  

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

In [ ]:
for bins in [3, 5, 7, 10]:
    y_binned = pd.qcut(y, q=bins, labels=False, duplicates= 'drop')
    scores = cross_val_score(clf_pipeline, X, y_binned, cv=5, scoring='accuracy')
    print(f"{bins} bins → Accuracy: {np.mean(scores):.4f}")

3 bins → Accuracy: 0.7390
5 bins → Accuracy: 0.6545
7 bins → Accuracy: 0.6385
10 bins → Accuracy: 0.5947


In [ ]:
df_test = pd.read_csv('test.csv')
# Bin original y (for training and mapping reference)
y_binned = pd.qcut(y, q=num_bin, labels=False, duplicates='drop')

# Now, get the bin intervals from the dtype of the binned series
bin_intervals = pd.qcut(y, q=num_bin, duplicates='drop').dtype.categories

bin_midpoints = [interval.mid for interval in bin_intervals]

# Predict bin labels on unseen/test data
y_pred_bins = grid_clf.predict(df_test) 

# Convert bin label to SHU approximation using bin midpoints
y_pred_shu = [bin_midpoints[int(bin_label)] for bin_label in y_pred_bins]

y_test_kaggle = pd.DataFrame(y_pred_shu, columns=["Scoville Heat Units (SHU)"])
y_test_kaggle.index.name = "index"
y_test_kaggle[['Scoville Heat Units (SHU)']].to_csv("kaggle.csv")

In [ ]:
bin_intervals

IntervalIndex([      (-0.001, 1687.746],    (1687.746, 11696.862],
                 (11696.862, 21014.122],    (21014.122, 33524.02],
                  (33524.02, 44015.419],   (44015.419, 55603.204],
                 (55603.204, 69801.462],   (69801.462, 81131.709],
                 (81131.709, 92165.908],  (92165.908, 104195.538],
               (104195.538, 115786.399],  (115786.399, 130505.42],
                (130505.42, 144631.227], (144631.227, 156024.772],
               (156024.772, 172074.327], (172074.327, 181797.978],
               (181797.978, 195615.438],  (195615.438, 223328.71],
                (223328.71, 242854.924], (242854.924, 264765.219],
               (264765.219, 290983.956], (290983.956, 325945.179],
               (325945.179, 377966.343],  (377966.343, 527639.86]],
              dtype='interval[float64, right]')

**Comment**: 

Accuracy vs. Number of Bins

As the number of bins decreases (e.g., from 10 to 3), classification accuracy tends to increase. This is because it's easier for the classifier to correctly predict broader categories. For example, since a large portion of the dataset consists of peppers with a SHU of 0, the classifier can perform well simply by assigning many samples to the first bin (which includes 0). This boosts accuracy but oversimplifies the predictions.


Impact on SHU Prediction Error (MAE or MSE)

However, when we convert bin predictions back into SHU values (e.g., using the midpoint of each interval), this coarse binning becomes problematic. If the first bin covers a wide range, say SHU 0 to 1600, and its midpoint is around 800, then all predictions in that bin are assigned ~800 — even though many actual values are 0. This creates large errors in SHU estimation (e.g., MAE or MSE).


Effect of Increasing Number of Bins

Using more bins creates finer granularity, allowing the model to make more precise distinctions between SHU ranges. This reduces the error when mapping bins back to SHU values. However, as the number of bins increases too much (e.g., beyond 100), bins may become too narrow, class imbalance increases, and the classifier may struggle. As a result, SHU prediction error (e.g., on Kaggle) stops improving significantly beyond a certain point.

### (C) A two-step analysis (two sequential pipelines)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_mapped = np.where(y_train > 0, 1, 0)

In [ ]:
class_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])


param_grid1 = {
'classifier__n_estimators': [100, 200],
'classifier__max_depth': [10, 20],
'classifier__min_samples_split': [5, 10],
'classifier__min_samples_leaf': [2, 4],
}

gs_class = GridSearchCV(class_pipe, param_grid1, cv=5, scoring='accuracy')
gs_class.fit(X_train, y_train_mapped)


best_classifier = gs_class.best_estimator_
class_pred = best_classifier.predict(X)

In [ ]:
spicy_peppers_indices = np.where(class_pred == 1)[0]

X_reg = X.iloc[spicy_peppers_indices]
y_reg = y.iloc[spicy_peppers_indices]

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42, stratify=y)

ValueError: Found input variables with inconsistent numbers of samples: [438, 935]

In [ ]:
reg_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_grid2 = {
'regressor__n_estimators': [100, 200],
'regressor__max_depth': [10, 20],
'regressor__min_samples_split': [5, 10],
'regressor__min_samples_leaf': [2, 4],
}

gs_reg = GridSearchCV(reg_pipe, param_grid2, cv=5, scoring= 'neg_mean_squared_error')
gs_reg.fit(X_reg_train, y_reg_train)

best_regressor = gs_reg.best_estimator_
reg_pred = best_regressor.predict(X_reg)

NameError: name 'X_reg_train' is not defined

In [28]:
shu_pred = pls_model.predict(test_df)
y_test_kaggle = pd.DataFrame(shu_pred, columns=["Scoville Heat Units (SHU)"])
y_test_kaggle.index.name = "index"
y_test_kaggle[['Scoville Heat Units (SHU)']].to_csv("kaggle1.csv")